In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f'''
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.f_store_open_order_sft 
AS
WITH DAT AS (
    SELECT
        dc_id,
        store_id,
        sku product_id,
        DATE(calday) calday,
        sft_so_processing_qty,
        rkt_sto_instock_qty,
        IFNULL(rkt_sto_instock_qty, 0) + IFNULL(sft_so_processing_qty, 0) open_order
    FROM
        {catalog_name}.udp_wcm_silver_realtime.sft_store_po_pending_inv
    WHERE
        client_code = 'WIN' AND
        type = 'OPENING_STOCK'
    QUALIFY ROW_NUMBER() OVER (PARTITION BY dc_id, store_id, sku, calday, type ORDER BY proc_date DESC) = 1
)
SELECT
    dc_id,
    store_id,
    product_id,
    calday,
    SUM(sft_so_processing_qty) sft_so_processing_qty,
    SUM(rkt_sto_instock_qty) rkt_sto_instock_qty,
    SUM(open_order) open_order
FROM DAT
GROUP BY 1,2,3,4;''')